## Transmission Economic Assessment with PyPSA-USA


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Stanford-Sustainable-Systems-Lab/CEE_272R_spring_2025/blob/main/HM6/HM6_starter_notebook.ipynb)

CEE 272R Spring 2024 Homework 6 \\
Assigned on 5/15/25, due at 11:59pm on Thursday, 5/22 \\
Written by Kamran Tehranchi & edited by Sonia, Mateus, and Fletcher

This assignment is set up to run in Google Colab, although you may install PyPSA and change file locations to run locally if you wish.

When you submit to Gradescope, please download the export the notebook, including all code outputs, as a pdf.

You should fill in all code blocks labeled "#TODO".

You are given a simplified network of the PyPSA-USA WECC network with the following characteristics:
- 3 buses
- 3 lines
- 167 Generators (MW), listed below by fuel type
```
    CCGT          56915.400
    OCGT          35345.100
    coal          24395.000
    geothermal     3914.200
    hydro         52844.738
    nuclear        7732.600
    oil            1100.300
    onwind        28557.900
    solar         29108.000
```
- Peak Load (MW):
```
    CA    49420.22
    PNW   17931.67
    SW    30590.69
```
- Time-series data for 12 months Jan - Dec 2019

Let's first install PyPSA and import the WECC network so we can visualize it. Run the next few code blocks to generate a plot of the nodes and existing lines.

Install dependencies

In [1]:
# !pip install pypsa highspy matplotlib cartopy xarray==2023.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.7/207.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 14.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 12.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 14.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 13.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Import the network

In [10]:
import pypsa
import matplotlib.pyplot as plt
## Download the file 'wecc_homework6.nc' and upload it to the 'files' folder in Google Colab
network = pypsa.Network('wecc_homework6.nc')


ModuleNotFoundError: No module named 'xarray.namedarray'

Plot a network visualization

In [ ]:
from pypsa.plot import add_legend_patches
import cartopy.crs as ccrs
import random

carriers = network.generators.carrier.unique()
colors = ["#%06x" % random.randint(0, 0xFFFFFF) for _ in carriers]
network.madd("Carrier", carriers, color=colors)

fig = plt.figure()
ax = plt.axes(projection=ccrs.EqualEarth())
capacities = network.generators.groupby(["bus", "carrier"]).p_nom.sum()

network.plot(
    ax=ax,
    bus_sizes=capacities / 2e5,
    margin=0.2
)

add_legend_patches(ax, colors, carriers)

As you can see, this network is somewhat similar to problem 3 of homework 5 with the California network, but here we actually have realistic generator and load data from 2019.

In this homework, we are going to go through the steps of assessing whether is makes economic sense to build an additional 230kV transmission line between California and the Southwest.

To do this, we are going to compare load costs and adjusted production costs for the different regions in the model. This will help tell a story of who, if any stakeholders, profit from a new transmission line and who, if any, lose money from it.

The goal of this assignment is to practice some Python coding, learn how to run a PyPSA network model and use some PyPSA functions, and analyze the results in the context of what we've learned about transmission grids.


**Problem 1** (3 points)

Simulate a sequential DCOPF of the current given network (i.e. simulate the network operation).

Relevant documentation: https://pypsa.readthedocs.io/en/latest/components.html#network

a)

In [ ]:
#TODO


Create a labeled time-series plot of the Locational Marginal Prices (LMPs) for the first week of January, 2019.

Relevant documentation: https://pypsa.readthedocs.io/en/latest/components.html#bus

b)

In [ ]:
#TODO


Run the code block below to get a visualization of the dispatch by fuel type for the first week

In [ ]:
#Dispatch plot of first week
nhours = 24*7
fig, ax = plt.subplots(figsize=(20, 5))
p_by_carrier = network.generators_t.p.groupby(network.generators.carrier, axis=1).sum().div(1e3).iloc[:nhours]
p_by_carrier.plot(
    kind="area",
    ax=ax,
    linewidth=0,
    cmap="tab20b",
)
ax.legend(ncol=5, loc="upper left", frameon=False)
ax.set_ylabel("GW")

**Problem 2** (4 points)
Next we will calculate Load Costs and Adjusted Production Cost (APC) for California and then the rest of WECC prior to expanding the line. Report all costs in Millions of Dollars.


You will write the code to calculate load costs yourself, but we provide the APC model. If you're curious, the model is below:
\begin{align*}
\text{Regional Adjusted Production Cost (APC)} &= \text{Production Cost} + \text{Purchases} - \text{Sales} &\\
\text{Production Cost} &= \sum_t^T \sum_g^G ( \text{P}_{g,t} \times \text{O}_{g,t} ) &\\
\text{Purchases} &= \sum_t^T (\text{Imports}_t \times \text{Load Weighted LMP}_t) &\\
\text{Sales} &= \sum_t^T (\text{Exports}_t \times \text{Generation Weighted LMP}_t) &\\
\text{Load Weighted LMP}_t &= \frac{\sum_{n} \text{Load Costs}_{n,t} }{\sum_{n} \text{Load}_{n,t}}  \quad \forall \quad n \in region &\\
\text{Generation Weighted LMP}_t &= \frac{\sum_{n} \text{Generation Revenue}_{n,t} }{\sum_{n} \text{Generation}_{n,t} } \quad \forall \quad n \in region &\\
\quad &\\
\text{where} \quad
\text{P: Active power generation of Generator g} &\\
\text{O: Operational/Marginal Cost of Generator g} &
\end{align*}


First, calculate the load costs for California only, reporting one value in millions of $. You'll first write a line of code to find all load costs, then report for CA only.

a)

In [ ]:
# TODO: calculate load costs for CA (fill in both variables below)
load_costs =
load_costs_ca =
print('Load Costs CA [millions of $]: ', load_costs_ca / 1e6)

Run the code below to calculate APC for California:

In [ ]:
# Calculate APC for California, report one #:
exports = network.lines_t.p0.where(network.lines_t.p0 > 0, 0) #
imports = network.lines_t.p0.where(network.lines_t.p0 < 0, 0) * -1
generator_production_by_node = network.generators_t.p.groupby(network.generators.bus, axis=1).sum()
# Production Cost
production_costs = network.generators_t.p * network.generators.marginal_cost
production_costs_ca = production_costs.groupby(network.generators.bus, axis=1).sum()['CA']

# Purchases
load_weighted_lmp_ca = load_costs['CA'] / network.loads_t.p_set['CA']
generator_revenue_ca = generator_production_by_node['CA'] * network.buses_t.marginal_price['CA']
gen_weighted_lmp_ca = generator_revenue_ca / generator_production_by_node['CA']

purchases_ca = imports['CA-PNW'] * load_weighted_lmp_ca + imports['CA-SW'] * load_weighted_lmp_ca
sales_ca = exports['CA-PNW'] * gen_weighted_lmp_ca + exports['CA-SW'] * gen_weighted_lmp_ca

APC_ca = production_costs_ca + purchases_ca - sales_ca

print("Adjusted Production Cost: ", APC_ca.sum() / 1e6)
print("Production Cost: ", production_costs_ca.sum() / 1e6)
print("Sales: ", sales_ca.sum() / 1e6)
print("Purchases: ",purchases_ca.sum() / 1e6)


Next, calculate load costs for WECC *not including California* prior to expanding the line.

b)


In [ ]:
# TODO: calculate load Costs for Non-CA, report one number in millions of $:
load_costs_wecc =
print('Load Costs [millions of $]: ', load_costs_wecc / 1e6)


Run the code below to calculate APC for WECC not including California:

In [ ]:
#Calculate APC for Non-CA:
# Production Cost
production_costs = network.generators_t.p * network.generators.marginal_cost
production_costs_wecc = production_costs.groupby(network.generators.bus, axis=1).sum()[['PNW','SW']].sum().sum()

# Purchases
load_weighted_lmp_wecc = load_costs[['PNW','SW']].sum(axis=1) / network.loads_t.p_set[['PNW','SW']].sum(axis=1)
generator_revenue_wecc = generator_production_by_node[['PNW','SW']] * network.buses_t.marginal_price[['PNW','SW']]
generator_revenue_wecc = generator_revenue_wecc.sum(axis=1)
gen_weighted_lmp_wecc = generator_revenue_wecc / generator_production_by_node[['PNW','SW']].sum(axis=1)

purchases_wecc = exports['CA-PNW'] * load_weighted_lmp_wecc + exports['CA-SW'] * load_weighted_lmp_wecc
sales_wecc = imports['CA-PNW'] * gen_weighted_lmp_wecc + imports['CA-SW'] * gen_weighted_lmp_wecc

APC_wecc = production_costs_wecc + purchases_wecc.sum() - sales_wecc.sum()

print("Adjusted Production Cost: ", APC_wecc.sum() / 1e6)
print("Production Cost: ", production_costs_wecc.sum() / 1e6)
print("Sales: ", sales_wecc.sum() / 1e6)
print("Purchases: ",purchases_wecc.sum() / 1e6)

**Problem 3** (3 points) Add the line to the network and re-simulate the network.

a)

In [ ]:
line_name = "CA-SW_newline"
bus0 = 'CA'
bus1 = 'SW'
reactance = 0.1 # Ohm
transfer_capacity = 2000 # MW (s_nom)

In [ ]:
# TODO: Add the line by filling in the missing parameters here:
network.add(
    "Line",
    line_name,
    # insert,
    # insert,
    # insert,
    # insert,
)

b)

In [ ]:
#TODO: simulate network with new line


**Problem 4** (4 points)

Calculate the new load costs and APC for California

a)

In [ ]:
#TODO - set load_costs and load_costs_ca_new variables
load_costs =
load_costs_ca_new  =
print('Load Costs CA [millions of $]: ', load_costs_ca_new / 1e6)


In [ ]:
# Calculate APC for California, report one #:
# Production Cost
exports = network.lines_t.p0.where(network.lines_t.p0 > 0, 0) #
imports = network.lines_t.p0.where(network.lines_t.p0 < 0, 0) * -1
generator_production_by_node = network.generators_t.p.groupby(network.generators.bus, axis=1).sum()
production_costs = network.generators_t.p * network.generators.marginal_cost
production_costs_ca = production_costs.groupby(network.generators.bus, axis=1).sum()['CA']

# Purchases
load_weighted_lmp_ca = load_costs['CA'] / network.loads_t.p_set['CA']
generator_revenue_ca = generator_production_by_node['CA'] * network.buses_t.marginal_price['CA']
gen_weighted_lmp_ca = generator_revenue_ca / generator_production_by_node['CA']

purchases_ca = imports['CA-PNW'] * load_weighted_lmp_ca + imports['CA-SW'] * load_weighted_lmp_ca + imports['CA-SW_newline'] * load_weighted_lmp_ca
sales_ca = exports['CA-PNW'] * gen_weighted_lmp_ca + exports['CA-SW'] * gen_weighted_lmp_ca + exports['CA-SW_newline'] * gen_weighted_lmp_ca

APC_ca_new = production_costs_ca + purchases_ca - sales_ca

print("Adjusted Production Cost: ", APC_ca_new.sum() / 1e6)
print("Production Cost: ", production_costs_ca.sum() / 1e6)
print("Sales: ", sales_ca.sum() / 1e6)
print("Purchases: ",purchases_ca.sum() / 1e6)

Repeat for non-CA WECC load costs and APC

b)

In [ ]:
# TODO: calculate load Costs for Non-CA, report one number in millions of $:
load_costs_wecc_new =
print('Load Costs [millions of $]: ', load_costs_wecc / 1e6)


In [ ]:
#Calculate APC for Non-CA:
# Production Cost
production_costs = network.generators_t.p * network.generators.marginal_cost
production_costs_wecc = production_costs.groupby(network.generators.bus, axis=1).sum()[['PNW','SW']].sum().sum()

# Purchases
load_weighted_lmp_wecc = load_costs[['PNW','SW']].sum(axis=1) / network.loads_t.p_set[['PNW','SW']].sum(axis=1)
generator_revenue_wecc = generator_production_by_node[['PNW','SW']] * network.buses_t.marginal_price[['PNW','SW']]
generator_revenue_wecc = generator_revenue_wecc.sum(axis=1)
gen_weighted_lmp_wecc = generator_revenue_wecc / generator_production_by_node[['PNW','SW']].sum(axis=1)

purchases_wecc = exports['CA-PNW'] * load_weighted_lmp_wecc + exports['CA-SW'] * load_weighted_lmp_wecc + exports['CA-SW_newline'] * load_weighted_lmp_wecc
sales_wecc = imports['CA-PNW'] * gen_weighted_lmp_wecc + imports['CA-SW'] * gen_weighted_lmp_wecc + imports['CA-SW_newline'] * gen_weighted_lmp_wecc

APC_wecc_new = production_costs_wecc + purchases_wecc.sum() - sales_wecc.sum()

print("Adjusted Production Cost: ", APC_wecc_new.sum() / 1e6)
print("Production Cost: ", production_costs_wecc.sum() / 1e6)
print("Sales: ", sales_wecc.sum() / 1e6)
print("Purchases: ",purchases_wecc.sum() / 1e6)

Problem 5 (5 points) Now we'll conduct Cost Benefit Analysis to determine if the transmission line should be built. First, we assume we only care about the benefits within California.

Here are some assumptions:
- Weight 70\% benefits of load and 30\% benefits of APC
- Assume the same savings patterns occur over the lifetime of the transmission line
- Assume the capital cost of the transmission line is incurred overnight and there is no fixed or variable O&M for the line over its lifetime.
- Interest rate 5\%
- Transmission line has assumed lifetime of 50 years & overnight capital cost of 1e9


We will provide the function for annualized cost:

In [ ]:
#Calculate Annualized Cost of the line:
def annualized_cost(capital_cost, lifetime, interest_rate):
    return capital_cost * (interest_rate * (1 + interest_rate) ** lifetime) / ((1 + interest_rate) ** lifetime - 1)

To find the cost benefit for CA, first calculate savings between the no new line and new line case for load costs and APC, then apply the weightings provided above to those savings. Call the annualized cost function from above to find "cost_annual" and calculate the overall cost benefit.

Each "savings" variable should be a scalar.

a)

In [ ]:
#TODO: Cost Benefit Analysis
savings_ca_load =
savings_apc_ca =
economic_benefit_ca =
print("Load Savings CA: ", savings_ca_load / 1e6)
print("APC Savings CA: ", savings_apc_ca/ 1e6)
print("Annual Economic Benefit CA: ", economic_benefit_ca / 1e6)

cost_annual =
cost_benefit_ca = (cost_annual /1e6) + (economic_benefit_ca / 1e6)
print("Cost-Benefit CA: ", cost_benefit_ca)


Comment on what this means for CA. Negative values indicated decreases in costs.

b)

In [ ]:
#TODO
#comment on results

Now we'll analyze whether this answer changes if we include the costs/benefits of all of WECC.

Run the cost benefit analysis for non-WECC buses then find the total cost benefit.

c)

In [ ]:
#TODO: Cost Benefit Analysis:
savings_wecc_load =
savings_apc_wecc =
economic_benefit_wecc =
print("Load Savings WECC: ", savings_wecc_load / 1e6)
print("APC Savings WECC: ", savings_apc_wecc / 1e6)
print("Economic Benefit WECC: ", economic_benefit_wecc / 1e6)
print("Negative Savings values indicate decrease in cost \n")

cost_benefit_total = (cost_annual /1e6) + ((economic_benefit_wecc + economic_benefit_ca) / 1e6)
print("Total System Cost-Benefit: ", cost_benefit_total)
print("Total Cost increases for WECC due to increase in total production cost and load costs.")

Comment on these results. What does this say overall about adding a new line?

d)

In [ ]:
#TODO comment on all results

**Problem 6** (1 point) In the above method we used production cost simulation to estimate the benefits of the transmission expansion.
Now, we are going to let PyPSA-USA determine the optimal capacity of the line via use capacity expansion methods.

 You can assume linear relationship between capital cost of transmission and nominal capacity of the line.

What is the optimal capacity of the line? (There's just one line of code you have to add here.) Notice the s_nom_extendable feature; this allows the nominal complex power of the line to expand.

In [ ]:

network = pypsa.Network('wecc_homework6.nc')
network.add(
    "Line",
    line_name,
    bus0=bus0,
    bus1=bus1,
    x=reactance,
    s_nom=0,
    capital_cost= cost_annual / transfer_capacity,
    s_nom_extendable=True,
)

Rerun the network simulation:

In [ ]:
#TODO


Run the code below to print out all of the line capacities:

In [ ]:
network.lines.s_nom_opt